<a href="https://colab.research.google.com/github/Ishakasar/Recommender-engine/blob/main/Isha_Kasar_Team_2_Capstone_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Build a recommender engine that reviews customer ratings and purchase history to recommend items and improve sales. </u></b>

### Amazon.com is one of the largest electronic commerce and cloud computing companies.

### Just a few Amazon related facts:

### They lost $4.8 million in August 2013, when their website went down for 40 mins. They hold the patent on 1-Click buying, and licenses it to Apple. Their Phoenix fulfilment centre is a massive 1.2 million square feet. Amazon relies heavily on a Recommendation engine that reviews customer ratings and purchase history to recommend items and improve sales.


### This is a dataset related to over 2 Million customer reviews and ratings of Beauty related products sold on their website.

### It contains

* ### the unique UserId (Customer Identification),
* ### the product ASIN (Amazon's unique product identification code for each product),
* ### Ratings (ranging from 1-5 based on customer satisfaction) and
* ### the Timestamp of the rating (in UNIX time)

### This dataset contains product reviews and metadata from Amazon, including 142.8 million reviews spanning May 1996 - July 2014.

### This dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [3]:
df1=pd.read_csv('/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/ratings_Beauty.csv')

In [4]:
df=pd.read_json('/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/reviews_Beauty_5.json.gz', lines = True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198502 entries, 0 to 198501
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      198502 non-null  object
 1   asin            198502 non-null  object
 2   reviewerName    197116 non-null  object
 3   helpful         198502 non-null  object
 4   reviewText      198502 non-null  object
 5   overall         198502 non-null  int64 
 6   summary         198502 non-null  object
 7   unixReviewTime  198502 non-null  int64 
 8   reviewTime      198502 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.6+ MB


In [6]:
df.describe()

,overall,unixReviewTime
count,198502.000000,1.985020e+05
mean,4.190391,1.364450e+09
std,1.166580,3.841937e+07
min,1.000000,1.023840e+09
25%,4.000000,1.354493e+09
50%,5.000000,1.372205e+09
75%,5.000000,1.391213e+09
max,5.000000,1.406074e+09


In [7]:
df.isnull().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [8]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198502 entries, 0 to 198501
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      198502 non-null  object
 1   asin            198502 non-null  object
 2   reviewerName    197116 non-null  object
 3   helpful         198502 non-null  object
 4   reviewText      198502 non-null  object
 5   overall         198502 non-null  int64 
 6   summary         198502 non-null  object
 7   unixReviewTime  198502 non-null  int64 
 8   reviewTime      198502 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.6+ MB


In [10]:
df.describe()

,overall,unixReviewTime
count,198502.000000,1.985020e+05
mean,4.190391,1.364450e+09
std,1.166580,3.841937e+07
min,1.000000,1.023840e+09
25%,4.000000,1.354493e+09
50%,5.000000,1.372205e+09
75%,5.000000,1.391213e+09
max,5.000000,1.406074e+09


In [11]:
df.isnull().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [12]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [13]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [14]:
df.rename(columns={"reviewerID": "UserId", "asin": "ProductId"}, inplace=True)
df['Rating'] = df1['Rating']


In [15]:
df.shape

(198502, 10)

In [16]:
df.dropna(inplace = True)

In [17]:
df.shape

(197116, 10)

In [18]:
df.head()

,UserId,ProductId,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Rating
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014",5.0
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014",3.0
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013",5.0
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013",4.0
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013",1.0


In [19]:
df.drop(['reviewerName',	'helpful',	'reviewText',	'overall',	'summary',	'unixReviewTime',	'reviewTime'], axis = 1, inplace = True)

In [20]:
f'l'

'l'

In [21]:
users_interactions_count_df = df.groupby(['UserId', 'ProductId']).size().groupby('UserId').size()
print('# of users: %d' % len(users_interactions_count_df))

interaction_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['UserId']]
print('# of users with at least 5 interactions: %d' % len(interaction_df))

# of users: 22345
# of users with at least 5 interactions: 22110


In [22]:
users_interactions_count_df.head()

UserId
A00414041RD0BXM6WK0GX    6
A00473363TJ8YSZ3YAGG9    6
A00700212KB3K0MVESPIY    9
A0078719IR14X3NNUG0F     8
A01198201H0E3GHV2Z17I    7
dtype: int64

In [23]:
interaction_df.head()

,UserId
0,A00414041RD0BXM6WK0GX
1,A00473363TJ8YSZ3YAGG9
2,A00700212KB3K0MVESPIY
3,A0078719IR14X3NNUG0F
4,A01198201H0E3GHV2Z17I


In [24]:
interactions_from_selected_users_df = df.merge(interaction_df, 
               how = 'right',
               left_on = 'UserId',
               right_on = 'UserId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions from users with at least 5 interactions: 196257


In [25]:
interactions_from_selected_users_df.head(20)

,UserId,ProductId,Rating
0,A00414041RD0BXM6WK0GX,B007IY97U0,5.0
1,A00414041RD0BXM6WK0GX,B00870XLDS,1.0
2,A00414041RD0BXM6WK0GX,B008MIRO88,5.0
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,5.0
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,5.0
5,A00414041RD0BXM6WK0GX,B00HFP4JZU,5.0
6,A00473363TJ8YSZ3YAGG9,B000052YQU,5.0
7,A00473363TJ8YSZ3YAGG9,B0019LVFI0,5.0
8,A00473363TJ8YSZ3YAGG9,B001L2BEWE,5.0
9,A00473363TJ8YSZ3YAGG9,B0020HEBX8,5.0


In [26]:
df.shape

(197116, 3)

In [27]:
interactions_from_selected_users_df.shape

(196257, 3)

In [28]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df.groupby(['UserId', 'ProductId'])['Rating'].sum().apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head()

# of unique user/item interactions: 196257


,UserId,ProductId,Rating
0,A00414041RD0BXM6WK0GX,B007IY97U0,2.584963
1,A00414041RD0BXM6WK0GX,B00870XLDS,1.000000
2,A00414041RD0BXM6WK0GX,B008MIRO88,2.584963
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,2.584963
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,2.584963


In [29]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['UserId'], 
                                   test_size=0.30,
                                   random_state=42)

In [30]:

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 137379
# interactions on Test set: 58878


##Creating pivot table

In [31]:
#Creating a sparse pivot table with users in rows and items in columns
pivot_matrix_df = interactions_train_df.pivot(index='UserId', 
                                                          columns='ProductId', 
                                                          values='Rating').fillna(0)

In [32]:
pivot_matrix_df.shape

(22110, 12090)

In [33]:
df.shape

(197116, 3)

In [34]:
pivot_matrix_df.head()

ProductId,7806397051,9759091062,9788072216,9790790961,9790794231,B00004TMFE,B00004TUBL,B00004TUBV,B00004U9UY,B00004U9V2,B000050B6U,B000052WY7,B000052WYD,B000052WYL,B000052WYN,B000052X9G,B000052XIA,B000052XW5,B000052XYQ,B000052XZP,B000052XZX,B000052Y25,B000052Y33,B000052Y5F,B000052Y6Z,B000052YJC,B000052YJD,B000052YJH,B000052YJM,B000052YJX,B000052YKI,B000052YKM,B000052YKS,B000052YKY,B000052YL1,B000052YLX,B000052YM0,B000052YM3,B000052YM4,B000052YM7,...,B00KFM16OW,B00KGDRCZW,B00KH6F6TM,B00KHGIK54,B00KHH2VOY,B00KI5UL8I,B00KIEGBHE,B00KK1V28I,B00KK62714,B00KKKW03U,B00KLA4INE,B00KLJDYL2,B00KLZO2JE,B00KNIL36E,B00KNJOLSA,B00KPKIV8I,B00KQ4PEBU,B00KQBR9FM,B00KQVTX06,B00KTAJAIY,B00KTLBDYM,B00KTLBEEQ,B00KTP8Q1G,B00KVE7R0Q,B00KVO2JWC,B00KWE08Q0,B00KXG8JF4,B00L0C529Q,B00L2K53BS,B00L2KVF9W,B00L3K91OW,B00L3LB0IG,B00L3LBUGI,B00L5BXG36,B00L5JHZJO,B00L5KTZ0K,B00L6Q3BH6,B00LCEROA2,B00LG63DOM,B00LLPT4HI
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A00414041RD0BXM6WK0GX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00473363TJ8YSZ3YAGG9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00700212KB3K0MVESPIY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0078719IR14X3NNUG0F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01198201H0E3GHV2Z17I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
pivot_matrix = pivot_matrix_df.values
pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
users_ids = list(pivot_matrix_df.index)
users_ids[:10]

['A00414041RD0BXM6WK0GX',
 'A00473363TJ8YSZ3YAGG9',
 'A00700212KB3K0MVESPIY',
 'A0078719IR14X3NNUG0F',
 'A01198201H0E3GHV2Z17I',
 'A02155413BVL8D0G7X6DN',
 'A02849582PREZYNEI31CV',
 'A029527620Q3SK5XW16RR',
 'A03236882LUP4ARMDDMXU',
 'A03364251DGXSGA9PSR99']

##SVD

In [40]:
X = pivot_matrix_df.T

In [60]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=20)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(12090, 20)

In [64]:
decomposed_matrix


array([[ 6.74436294e-04,  4.53218145e-03,  6.69908572e-03, ...,
        -7.90975880e-03,  3.79400990e-03,  3.18556221e-03],
       [ 5.66019152e-03,  6.46088523e-04,  2.66212462e-02, ...,
        -3.98590296e-02,  7.34700557e-03, -1.07666244e-02],
       [ 2.76367094e-03,  1.50242357e-03,  6.36056560e-03, ...,
        -2.11872387e-03, -8.66262769e-04, -2.85117374e-02],
       ...,
       [ 7.65229736e-04,  4.85338790e-03,  8.77447207e-05, ...,
         1.21747569e-04,  4.59305728e-04, -2.20809356e-05],
       [ 6.75154758e-03,  9.59841444e-02, -3.41020078e-03, ...,
         3.87449390e-03,  9.09993987e-03,  1.10790454e-02],
       [ 1.10302161e+00,  2.27604998e-01, -4.29551299e-02, ...,
         9.21874648e-02, -1.46555237e-01, -1.36540448e-01]])

In [61]:
corr_mat = np.corrcoef(decomposed_matrix)
corr_mat.shape

(12090, 12090)

In [62]:
def recbyindex():
  i = int(input())
  product_names = list(X.index)
  product_ID = product_names.index(product_names[i])
  print('Product bought:',product_names[i])
  corr_id = corr_mat[product_ID]
  rec = list(X.index[corr_id > 0.80])
  rec.remove(product_names[i]) 
  print('Also recommended:\n',rec[0:10],'\n')



In [63]:
recbyindex()

20
Product bought: B000052XZX
Also recommended:
 ['9759091062', 'B00005NFBD', 'B00006IV22', 'B00007KQF3', 'B0000C4COV', 'B0001ZYLAO', 'B00027DMJW', 'B00028MLG6', 'B0002G21GI', 'B0002JG2NI'] 

